# **Summary** 📘

1.   **Abstract**
2.   **Introduction**
3.   **Objectives**
4.   **Methodology**
4.   **Model**
4.   **Using the Model**
5.   **Optimizing the Model**
5.   **Visualizing Optimized Parameters**
5.   **Results**
6.   **References**

## **Benchmarking Tool for Thin-Film Transistors** ⚡

## **Student**:

Rodrigo Santos Batista – rsb6@cin.ufpe.br

CIn ­ UFPE

## **Research project**:

Benchmarking emerging technologies for near and far computing

inside sensors (BEeTle)

## **Coordinator**:

Stefan Michael Blawid – sblawid@cin.ufpe.br

CIn ­ UFPE

### **Abstract** 📩

> Thin-film transistors (TFTs) are electronic switches where a current flows from the source terminal to the drain terminal, controlled by the voltage applied to the third terminal, the gate. Benchmark models need to reasonably reproduce the current-voltage (IV) characteristics of TFTs with a minimal number of parameters. Here, "reasonably" means that the model parameters can be reliably and unambiguously extracted from experimental IV curves.

### **Introduction** 📢


> The ubiquitous computing paradigm envisions computing appearing everywhere at any time. Flexible electronics are considered an essential enabling technology `[1]`. Flexible electronics offer various form factors, including soft, flexible, stretchable, and lightweight forms, thus allowing ambient intelligence to become widely available. Promising applications in health, industrial, environmental, agricultural, and structural monitoring have already been demonstrated `[2]`. It is generally accepted that the low cost required for edge devices `[3]` of future Internet of Things (IoT) infrastructures can only be achieved through printing manufacturing. The basic printed blocks of flexible electronics include sensors, energy harvesters, displays, and antennas. Many of these components require thin-film transistors (TFTs) as active switches. TFTs are usually compared across different devices and technologies based on threshold voltage, charge carrier mobility, and series resistance. However, conventional parameter extraction established for silicon transistors often leads to misleading results when applied to TFTs `[4]`. Some of the peculiarities of TFTs can be addressed by the virtual source (VS) emission-diffusion (ED) theory `[5]`. In combination with nonlinear least squares solvers, VSED-based benchmark models can replace conventional extraction methods and are applicable to a wide range of TFT technologies `[6]`.

### **Objective** 🔎


> The optimization of parameters for TFT devices is an important process in the manufacturing of thin-film electronics. The main reason for performing this process is to improve the performance of the devices, making them more efficient, reliable, and cost-effective. Essentially, we want to optimize certain parameters embedded in a model that will provide the best results for the production of these devices. A simplified form of how the model operates can be described as follows: `ID = F(VGS, VDS)` where:
> * ID → Electric current that flows between the source and drain terminals of a thin-film transistor (TFT)
> * VGS → Voltage applied to the gate terminal of the transistor, which controls the current flowing between the source and drain terminals
> * VDS → Voltage applied to the drain terminal of the transistor, which determines the current flowing between the source and drain terminals when the gate voltage is kept constant.

> VGS and VDS are important for the operation of the TFT transistor and for the optimization of its parameters, which can be obtained from the device's transfer and output curves. From these operations, it is possible to obtain information about its parameters, such as Vtho, l, n, JD, leak, δ, RS, λ, and Vcrit. Since these parameters are limited in the model, it allows for the simultaneous adjustment of all values. Therefore, parameter optimization is performed using a nonlinear least squares solver, which allows for efficient and unambiguous parameter adjustment. In this way, it is possible to gain a better understanding of the TFT device's behavior and optimize its operation for specific applications. Additionally, parameter optimization is also important to ensure the quality and reliability of the device, as well as its integration into complex electronic circuits.

### **Methodology** 📚


> To perform this optimization, we will use the SciPy library, an open-source Python module that provides advanced mathematical functions and algorithms for scientific data processing and numerical analysis. More specifically, we will apply the `curve_fit` function, which is a type of optimization that finds an ideal set of parameters for a defined function that best fits a given set of observations.

> Unlike supervised learning, curve fitting requires the author to define the function that maps input examples to outputs. The mapping function, also called the base function, can take any form, including a straight line (linear regression), a curved line (polynomial regression), and many others. This provides flexibility and control to define the shape of the curve, where an optimization process is used to find the optimal specific parameters of the function. For our purposes, this will be to optimize the parameters of the function $ID = F(VGS, VDS)$, which will give us the current at each pair of points from the experiment conducted.

> Here is a trivial example of how `curve_fit` works:

  > * First, the curve is constructed.
  > * Then, we apply the Python curve-fitting API.

> In this way, we will think of curve fitting in two dimensions, like a 2D graph. So, let's consider that data collection was performed from the problem domain with inputs and outputs.

> The `x-axis is the independent variable` or the input to the function. The `y-axis is the dependent variable` or the output of the function. We do not know the shape of the function that maps input examples to outputs, but we suspect we can approximate the function with a standard function shape. Curve fitting involves first defining the `functional form of the mapping function` (base function or objective function) and searching for the parameters in the function that result in the minimum error.

> The error is calculated using observations from the domain and passing the inputs to our candidate mapping function, calculating the output, and comparing the calculated output with the observed output.

> Once in shape, we can use the mapping function to interpolate or extrapolate new points in the domain. It is common to run a sequence of input values through the mapping function to calculate a sequence of outputs, then create a line graph of the result to show how the output varies with the input and how well the line fits the observed points.

> The key to curve fitting is the shape of the mapping function. A straight line between inputs and outputs can be defined as follows: `y = a * x + b`
Where `y is the calculated output`, `x is the input`, and `a` and `b` are parameters of the mapping function found using an optimization algorithm. This is called a linear equation because it is a weighted sum of the inputs. In a linear regression model, these parameters are referred to as coefficients; in a neural network, they are called weights.

> This equation can be generalized to any number of inputs (which is the purpose for us), meaning that the notion of curve fitting is not limited to two dimensions (one input and one output) but could have many input variables.

> An important observation is that adding arbitrary mathematical functions to our mapping function usually means we cannot calculate the parameters analytically and instead need to use an iterative optimization algorithm. This is called `nonlinear least squares`,
as the objective function is no longer convex (not linear), and it is not as simple to solve.

> Since our model's data is bi-dimensional, we are not in the trivial case. Thus, when passing data to the model, we will need to preprocess it first. For instance, suppose that for our problem, we will have the voltages Vd and Vg as a set of vectors. Essentially, what will be computed is an $MxN$ matrix; however, the `curve_fit` function only calculates ideal parameters for the one-dimensional vector case. So, we will first allocate all Vg voltages into a matrix and do the same for Vd voltages. Then, we will use Python's `np.ravel()` function to transform an array into a one-dimensional array. This operation does not affect the integrity of the data, only resizes it. The `np.ravel()` function returns a copy of the original array, with all elements of the original array "flattened" into a single one-dimensional array. This means that the function returns an array with all the rows of the original array concatenated into a single row.

### **Model Description** 🔧


The mathematical model to which we will subject the experimental data consists of a set of equations, where each variable has its degree of importance and contribution. Below is its prototyping as well as what each parameter signifies:

The drain current of a TFT consists of mobile charges $Q_{free}$ modulated by the gate voltage and moves with a velocity modulated by the drain voltage. At high electric fields from source to drain, the carrier velocity saturates at a value $V_{sat}$. However, carriers need to scale a potential barrier on the path from source to drain, which represents a bottleneck for carrier transport. The limited rate of charge injection from the top of the barrier potential choke point can be termed as a virtual source (VS). The modified $V_{sed}$ model suggests a specific form of drain current per gate width W in VS:
$$J_D = \frac{I_D}{W} = V_{sat} \cdot F_{sat} \cdot Q_{free} $$

Some charged particles are not mobile. In certain cases, an exponential tail of trapping states ranging from the valence band edge to the band gap can relate the free and total carrier density by a power law. This occurs because all states are occupied according to the same quasi-Fermi level, as seen in Equation (B4) in [8] and its earlier derivation. Thus, the equation shows that
$$Q_{free} = q \cdot \sigma_v \cdot \left(\frac{Q_{tot}}{q \cdot \sigma_{traps}}\right)^l$$

Here, $σ_V$ and $σ_{traps}$ represent the density of valence states and traps in a single semiconductor layer in $V_S$, respectively. The value of the exponent $l$ is determined by the effective "temperature" ratio defining the exponential distribution of trap states' energy and the device temperature, originating from the Boltzmann energy distribution of free charge carriers. However, since the exact trap distribution is generally unknown, $l$ is treated as a model parameter. The distinction between free and trapped charge carriers, as presented in Equation (2), is the main adaptation of the VSED framework for thin film materials proposed in this work.

For a depleted semiconductor, an exponential accumulation of holes (positive charges) is expected with increasing gate field, which ceases when substantial shielding by the accumulated charge sheet is established. The following phenomenological expression first proposed in [9] is employed:
$$ Q_{tot} = C_I \cdot n \cdot V_T \cdot \ln \left[ 1 + e^{\psi \cdot V_S - V_{GS}} \cdot n \cdot V_T \right],$$
$$ \psi \cdot V_S = V_{tho} + |\delta| \cdot V_{DS}$$

The gate insulator capacitance depends on the dielectric constant ε and the insulator thickness tI: $C_I = \frac{ε}{t_I}$. The thermal voltage is represented by $V_T = \frac{kT}{q}$. The transition from weak to strong accumulation is modeled by parameters $n, V_{th0}$, and $δ$, where $n$ is influenced by the charging of the semiconductor region adjacent to the gate insulator interface, filling surface states and affecting the band bending rate with gate bias.

The interface potential becomes independent of VGS when it reaches a bias value $ψ \cdot V_S$, also known as threshold voltage. This allows parameterizing the potential barrier control by gate bias in TFTs. The parameter $ψ \cdot V_S$ represents the critical bias at which the potential at $V_S$ becomes independent of gate voltage. During hole accumulation at the gate insulator interface, parameters n and $l$ describe the loading of trap states, and the voltage scale for the exponential increase of drain current with gate bias is given by $(\frac{n}{l}) \cdot V_T$ in the VSED model. The parameter $δ$ does not necessarily represent drain-induced barrier lowering (DIBL).

The charge injection velocity $Q_{free}$ in the transistor channel is represented by $V_{inj} = V_{sat} \cdot F_{sat}$. It is common to estimate the drain current in TFTs as a drift motion of injected charge carriers, but this can lead to a mistaken conclusion about saturation velocity $V_{sat}$. While in high electric field regions the saturation velocity equals the body semiconductor saturation velocity, in low electric field regions the saturation velocity is defined by the unidirectional thermal velocity of the quasi-Fermi potential, not the electrostatic potential. This concept is discussed in [10]:
$$V_{sat} = \frac{2D}{\bar{\lambda}_{free}} = \frac{2 \cdot \mu \cdot V_T}{\bar{\lambda}_{free}}$$

which is parametrized here by the diffusion coefficient $D$ and may relate to drift mobility $μ$ through the Einstein relation. In low drain bias, charge carrier diffusion is the dominant transport mechanism in much of the TFT and not just in $V_S$. The characteristic length in Eq. (5) will no longer be given by the mean free path $\bar{\lambda}_{free}$ but by the diffusion length. Since only carriers of a single type are injected into a basically intrinsic semiconductor, the diffusion length can be very large and comparable to the device dimension given by the gate length $L_G$. $F_{sat}$ introduces a length scale $λ = \frac{L_G}{\bar{\lambda}_{free}}$, which effectively replaces ${\bar{\lambda}_{free}}$ → $L_G$ under low drain bias.

The remaining function $F_{sat}$ describes the drainage of accumulated holes. For long-channel TFTs, diffusion and emission theory [11, 5] represent an interesting approach to determine $F_{sat}$:

$$F_{sat} = \frac{1}{1 + 2t} \cdot \frac{1-e^{-\left(\frac{-V_{SD}}{V_T}\right)}}{1+e^{-\left(\frac{-V_{SD}}{V_T}\right)}} \cdot \frac{1}{1+2t}$$

For drain bias significantly exceeding thermal voltage, the transition function is described by $F_{sat} = \frac{1}{1 + 2t}$. The critical probability factor $t$ is determined by a mean Boltzmann factor in the channel region controlled by the gate and can be obtained from the specific potential profile. For long-channel devices, an analytical form has been proposed, which can be found in equations (4) to (12) of reference [11].

$$ t = \frac{2 \cdot \lambda}{m^2 (1 - η^2)} \cdot \left[ (1 - m \cdot η) \cdot e^{-m (1 - η)} - (1 - m) \right]$$

$$ η = 1 - \tanh\left(\frac{V_{SD}}{mV_T}\right)$$

$$ m = \frac{2 \cdot \frac{V_{Gt}}{V_T}}{1 + \sqrt{\frac{2 \cdot V_{Gt}}{V_{crit}}}}$$

$$ V_{Gt} = \frac{Q_{tot}}{C_I}$$

Note that an increase in transistor overdrive leads to a spacious diffusion region and shifts the onset of saturation to higher drain bias. In high gate bias, the necessary increase in saturation voltage decreases and transforms into square root growth for $V_{Gt} > V_{crit}$. Note that the saturation velocity given by Eq. (5) is achieved only for both large $V_{SG}$ (low barrier) and large $V_{SD}$ (sink load). In general, the critical diffusion length replacing $\lambda_{free}$ in Eq. (5) is a fraction of $L_G$ dependent on gate voltage. The $V_{Gt}$ required to achieve maximum injection velocity given by unidirectional thermal velocity decreases with $L_G$.

* The first equation is used to calculate the average lifetime of an unstable particle. It relates the average lifetime ($t$) of an unstable particle to its mass ($m$), binding energy ($λ$), and damping factor ($η$). The equation is a mathematical expression of the exponential decay law of the particle.

* The second equation is used to calculate the reflection coefficient of a wave in a MOSFET transistor (metal-oxide-semiconductor field-effect transistor). The equation relates the reflection coefficient ($η$) to the potential difference between drain and source ($V_{SD}$), ambient temperature ($V_T$), and a device parameter ($m$).

* The third equation is used in semiconductor devices to calculate the slope factor of a MOSFET transistor. The slope factor is a measure of the transistor's efficiency in turning on and off quickly. The equation relates the slope factor ($m$) to the transistor's threshold voltage ($V_{GT}$), a critical voltage constant ($V_{crit}$), and ambient temperature ($V_T$).

* The fourth equation is used to calculate the threshold voltage of a MOSFET transistor. The threshold voltage is the minimum voltage required to turn on the transistor. The equation relates the threshold voltage ($V_{GT}$) to the total charge ($Q_{tot}$) stored on the transistor gate, the gate capacitance ($C$), and the current ($I$) applied to the gate.

Understanding how each equation relates and what each one consists of, we will abstract all this 'complex' mathematics into the model code and execute it to observe what kind of results it will yield.

Remembering that our goal here will be to optimize the coefficients $J_T, V_{tho}, δ, l

, n, λ$, and $V_{crit}$ used in the modeling of the equations.

* $J_T$ → Junction Temperature Coefficient. It is a parameter representing the temperature variation at the junction of an electronic device, such as a transistor, diode, or integrated circuit. The value of $J_T$ is important for the correct design and operation of these devices.

* $V_{tho}$ → Threshold Voltage. It is the minimum electric voltage that must be applied to an electronic device, such as a MOSFET transistor, to initiate current conduction. The value of $V_{tho}$ is determined by the properties of the semiconductor material used in the device construction.

* $δ$ → Diffusion Layer Thickness. It is a parameter representing the thickness of the doped semiconductor material layer deposited on the semiconductor substrate surface to form a transistor or other electronic device.

* $l$ → Diffusion Length. It is a parameter representing the distance that the semiconductor material doping diffuses on the semiconductor substrate surface during the electronic device manufacturing process.

* $n$ → Diode Ideality Factor. It is a parameter describing the relationship between the electric current and voltage in a diode. The value of $n$ is used to calculate the diode's voltage drop at different current levels.

* $λ$ → Voltage Drop Coefficient. It is a parameter describing the voltage drop in an electronic device relative to the electric current passing through it. The value of $λ$ is used to calculate the voltage drop in a MOSFET transistor at different current levels.

* $V_{crit}$ → Critical Voltage. It is an electric voltage representing the maximum voltage limit that an electronic device can withstand without damaging its structure or functionality. The value of $V_{crit}$ is determined by the properties of the semiconductor materials used in the device construction and is important for the safe design and operation of electronic circuits.

Remembering that our objective here will be to optimize the coefficients $J_T, V_{tho}, δ, l, n, λ$, and $V_{crit}$ used in the modeling of the equations.

* $J_T$ → Junction Temperature Coefficient. It is a parameter representing the temperature variation at the junction of an electronic device, such as a transistor, diode, or integrated circuit. The value of $J_T$ is important for the correct design and operation of these devices.

* $V_{tho}$ → Threshold Voltage. It is the minimum electric voltage that must be applied to an electronic device, such as a MOSFET transistor, to initiate current conduction. The value of $V_{tho}$ is determined by the properties of the semiconductor material used in the device construction.

* $δ$ → Diffusion Layer Thickness. It is a parameter representing the thickness of the doped semiconductor material layer deposited on the semiconductor substrate surface to form a transistor or other electronic device.

* $l$ → Diffusion Length. It is a parameter representing the distance that the semiconductor material doping diffuses on the semiconductor substrate surface during the electronic device manufacturing process.

* $n$ → Diode Ideality Factor. It is a parameter describing the relationship between the electric current and voltage in a diode. The value of $n$ is used to calculate the diode's voltage drop at different current levels.

* $λ$ → Voltage Drop Coefficient. It is a parameter describing the voltage drop in an electronic device relative to the electric current passing through it. The value of $λ$ is used to calculate the voltage drop in a MOSFET transistor at different current levels.

* $V_{crit}$ → Critical Voltage. It is an electric voltage representing the maximum voltage limit that an electronic device can withstand without damaging its structure or functionality. The value of $V_{crit}$ is determined by the properties of the semiconductor materials used in the device construction and is important for the safe design and operation of electronic circuits.





## **Using the Model** 💻

In [ ]:
!git clone https://github.com/RodrigoSantosB/Model_OTFT.git

Below, insert the address of the `MODULE_OTFT` module that contains the files to be imported. This folder contains all the necessary files that the model uses to function. It can be loaded following the example: `sys.path.append('/content/drive/MyDrive/your/path/Model_OTFT')`, simply replace the path below with the corresponding path from your mounted drive containing the module.

In [ ]:
import sys
sys.path.append('/content/Model_OTFT')

In [ ]:
# enter into Model_OTFT directory
%cd /content/Model_OTFT

In [ ]:
# Show the actually branch
!git branch

In [ ]:
# Switch for to Model_02 branch
!git checkout -f Model_02

In [3]:
# Load the used classes
from modules_otft._imports import *
from modules_otft._read_data import ReadData
from modules_otft._model import TFTModel
from modules_otft._grafics import  TFTGraphicsPlot
from modules_otft._menu import TFTMenu

# from modules_otft._global_vars import *
from modules_otft._config import *
from modules_otft._utils import *


---
## **Instantiating the used classes**
---


In [4]:
plot = TFTGraphicsPlot()
menu = TFTMenu()
read = ReadData()

---
## **Loading sample data** 📚


---



In [5]:
# @title ### **Load parameters local / remote directory.**
# @markdown In this section, you can load the intrinsic parameters of the model you want to optimize. The load can be done locally, by choosing your `json` file containing all the values ​​to be loaded, parameters `Vtho, delta, n, l, lam, Vgcrit, Jth, Rs`

# @markdown Note, also, that it is necessary to provide the respective values ​​of `resistance_scale`, `current_carry` and `with_transistor`, which are respectively: resistance values, current values ​​- where the transistor typically operates - and transistor width (associated technology). If you want to add a value other than the default, add it to the field. Remember that the values ​​in the `JTH` and `Rs` fields will be multiples of the values ​​defined in these fields and the value of `with_transistor` is in units of `[cm]`. So make sure you provide consistent values ​​so that the template output can also be consistent

# @markdown ---

# Read and show the path file Json
settings = enter_with_json_file()
print()
print()

ld_voltages = load_voltages(settings['loaded_voltages'])

type_curve_plot = get_type_plot(settings['type_curve_plot'])

shift_list = calculate_shift_list(settings['shift_volt_data'], ld_voltages, settings['curves_transfer'])

list_tension_shift = get_shift_list(read, settings['curves_transfer'], shift_list, ld_voltages)

path_voltages = read.read_files_experimental(settings['path'], list_tension_shift)

Vv, Id, input_voltage, n_points, count_transfer, count_output = read.load_data(settings['type_read_data_exp'], path_voltages, settings['current_typic'], 
                                                                               settings['experimental_data_scale_transfer'], 
                                                                               settings['experimental_data_scale_output'], type_curve_plot)


print("Shift value    : " + f'{shift_list}')
print()

print("List volt shift: " +  f'{list_tension_shift}')
print()
list_curves = menu.view_path_reads(path_voltages, list_tension_shift)

print()
print('The table above shows all the files that were read in the directory, if you want to filter files so that only those that are of interest are read, do this by passing numbers that correspond to the order of the experiment curves,')
print('separated by ,. For example: 0:tranfer-5v , 3:output-40v and so on')
print('\n\n')

path_voltages, list_tension_shift, list_tension, = filter_and_load_files(read, settings['path'], settings['select_files'], list_curves, list_tension_shift, ld_voltages)

print()

print("List volt shift  : " f'{list_tension_shift}' )
print("List tension     : " f'{list_tension}' )
print()
_ = menu.view_path_reads(path_voltages, list_tension_shift)
print('\n\n')


tp_tst = get_transistor_type(settings['type_of_transistor'])
resistance = get_resistance(settings['resistance_scale'])
current = get_current(settings['current_carry'])
width_t = get_transistor_width(settings['with_transistor'])
load_parameters = load_coefficients(settings['loaded_parameters'])


print()
menu.print_values(load_parameters)
print()

mode_idleak, load_idleak = load_idleak_parameters(settings['idleak_mode'], settings['loaded_idleak'])

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


___________________________________________________________________________________ SETTINGS PRESENT IN THE JSON FILE:___________________________________________________________________________________

| path: /home/rsb6/Desktop/Artigo JICS/Model_OTFT/datas/SbMicro
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| experimental_data_scale_transfer: mA
-----------------------------------------------------------------------------------------

In [6]:
# @title ##### ***GRAFICS PLOT OPTION:*** { run: "auto" }
option = 'show both curves' # @param ["Show Transfer curve", "Show output curve", "show both curves"]

plot = initialize_graphics_plot(settings['image_size_width'], settings['image_size_height'])
model = instance_model(read, TFTModel, n_points, type_curve_plot, load_parameters, input_voltage, Vv, load_idleak, width_t, count_transfer, tp_tst, settings['experimental_data_scale_transfer'], settings['current_typic'], resistance, current)
in_model_data, in_exp_data, out_model_data, out_exp_data = load_experimental_data(read, count_transfer, Vv, Id, model, count_output)
plot_curves(option, plot, list_tension, list_tension_shift, count_transfer, in_model_data, out_model_data, in_exp_data, out_exp_data, shift_list, settings['select_files'], settings['current_typic'], type_curve_plot)
read.compute_relative_distance(Id, model)








**************************************************************************************************************************************************

|RELATIVE ERROR BETWEEN CURVES: 2873.5874

**************************************************************************************************************************************************


### **Model Optimization** ⚡

This optimization section focuses on estimating the optimal parameters for a MOSFET transistor given a sequence of input voltage $V$ [V] and current $I$ [mA], using the mathematical model introduced in section [5]. The goal here is to estimate these optimal parameters using the `curve_fit` function, as also introduced in section [4] (for more details, refer to that section). The parameters in question are listed below:

- $V_{tho}$: Gate threshold voltage of the transistor.
- $\Delta$: Channel modulation coefficient.
- $N$: Number of transistor channels.
- $L$: Channel length of the transistor.
- $\Lambda$: Channel modulation length.
- $V_{crit}$: Transistor breakdown voltage.
- $J_{th}$: Transistor saturation current.
- $R_s$: Transistor series resistance.



In [7]:
# @title #### **OPTIMIZING THE MODEL.** { run: "auto" }
lw_bounds, up_bounds = get_bounds(settings['lower_bounds'], settings['upper_bounds'])
tolerance_factor = get_tolerance_factor(settings['tolerance_factor'])
tlr_factor = get_tolerance_factor(tolerance_factor)
lw_bounds, up_bounds = configure_bounds(settings['default_bounds'], lw_bounds, up_bounds)

model_id = create_model_opt(TFTModel, input_voltage, n_points, type_curve_plot, settings['current_typic'],
                            settings['experimental_data_scale_transfer'], load_idleak, mode_idleak, count_transfer,
                            resistance, current)

optimizer = create_optimizer(settings['current_typic'], settings['experimental_data_scale_transfer'], settings['experimental_data_scale_output'],
                             settings['type_read_data_exp'], path_voltages, type_curve_plot, settings['optimization_method'], (lw_bounds, up_bounds))

configure_optimizer(optimizer, settings['default_bounds'], tlr_factor)

coeff_opt, coeff_error, text_verbose = optimize_model(optimizer, model_id, load_parameters, *path_voltages)

# Exibição dos resultados
print('\n\n\n')
print('*' * 200)
print(text_verbose.capitalize())
print('*' * 200)
print('\n\n\n')


----------------------------------------------------------------------------------------------------
TRUST REGION REFLECTIVE (TRF) MODE
----------------------------------------------------------------------------------------------------





********************************************************************************************************************************************************************************************************
`ftol` termination condition is satisfied.
function evaluations 32, initial cost 2.4562e+06, final cost 1.3832e+04, first-order optimality 2.02e+02.

********************************************************************************************************************************************************************************************************






In [8]:
# @title #### **SHOW MODEL PARAMETERS OPTIMIZED.** { run: "auto" }
# @markdown ### **Loading Parameters and Optimizing the Model** ⚡
# @markdown > This section is dedicated to visualizing the parameters obtained through optimization. To do this, simply choose any of the options below:
# @markdown ##### ***Show the list coefficients optimized:***
# @markdown ##### **--> Vtho, delta, n, l, lam, Vgcrit, Jth, Rs**
# @markdown ---

option = 'Show the table of values opt' # @param ["Show the coeff values","Show the initial values", "Show the table of values opt", "coeff opt", "coeff error"]
show_model_parameters_optimized(menu, option, load_parameters, coeff_opt, coeff_error, settings['current_typic'], current, resistance)



---------------------------
 COEFICIENTES OTIMIZADOS:

╒════════════════════╤═════════════════╤═══════════════════╤══════════════════════════════════╕
│ Parâmetro          │   Valor Inicial │   Valor Otimizado │   Erro associado (desvio padrão) │
╞════════════════════╪═════════════════╪═══════════════════╪══════════════════════════════════╡
│ VTHO [V]           │       8         │          8        │                      0.234251    │
├────────────────────┼─────────────────┼───────────────────┼──────────────────────────────────┤
│ DELTA              │       0.010898  │          0.01649  │                      0.000974433 │
├────────────────────┼─────────────────┼───────────────────┼──────────────────────────────────┤
│ N                  │      34.773     │         27.1544   │                     15.896       │
├────────────────────┼─────────────────┼───────────────────┼──────────────────────────────────┤
│ L                  │       2.5641    │          2.43855  │                    

### **Comparing** ✅

---
This section aims to evaluate the model's performance relative to the initial data. Visual observation allows assessing whether the model significantly outperformed the initial coefficients.

In this section, simply execute the cells below; no explicit modifications are required.

---

In [9]:
# @title #### **Print grafics plots of model Optimizad and model data.** { run: "auto" }
# @markdown ### ***Choice the plot that you want to view:***
# @markdown ---
option = 'show both curves comp' # @param ["Show Transfer curve opt", "Show output curve opt", "show both curves opt", "Show Transfer curve comp", "Show output curve comp", "show both curves comp"]

model_opt = create_optimized_model(read, TFTModel, n_points, type_curve_plot, coeff_opt, input_voltage, Vv, load_idleak, width_t, count_transfer, tp_tst, settings['experimental_data_scale_transfer'], settings['current_typic'], resistance, current)

in_model_data_opt, in_exp_data_opt, out_model_data_opt, out_exp_data_opt, in_model_data_comp, in_exp_data_comp, out_model_data_comp, out_exp_data_comp = get_model_data(read, count_transfer, count_output, Vv, Id, model_opt, model, compare=True)

plot_curves(option, plot, list_tension, list_tension_shift, count_transfer, in_model_data_comp, out_model_data_comp, in_exp_data_comp, out_exp_data_comp, shift_list, settings['select_files'], settings['current_typic'], type_curve_plot, compare=True)

read.compute_relative_distance(Id, model_opt)
print('\n\n\n\n\n')








**************************************************************************************************************************************************

|RELATIVE ERROR BETWEEN CURVES: 5100.6423

**************************************************************************************************************************************************






